In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melbourne-housing-snapshot/melb_data.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

The main objective of this notebook is to experiment how piplining works

In [3]:
path = '../input/melbourne-housing-snapshot/melb_data.csv'
data = pd.read_csv(path)

In [4]:
y = data['Price']
X_ = data.drop(['Price'],axis=1)

In [5]:
X_train,X_val,y_train,y_val = train_test_split(X_,y, train_size=0.8,test_size=0.2)

In [6]:
categorical_col = [col for col in X_train.columns if X_train[col].nunique() < 10 and 
                   X_train[col].dtype=='object']
numerical_col = [col for col in X_train.columns if X_train[col].dtype in ['int64','float64']]

In [7]:
cols = categorical_col + numerical_col
X_train = X_train[cols].copy()
X_val = X_val[cols].copy()

In [8]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
7118,h,S,Southern Metropolitan,4,12.1,3163.0,4.0,2.0,1.0,650.0,NaN,1935.0,-37.89180,145.07690,4442.0
6286,h,SP,Northern Metropolitan,3,6.5,3071.0,2.0,2.0,1.0,417.0,159.0,1920.0,-37.75950,145.00170,8870.0
12944,h,SP,Northern Metropolitan,4,20.6,3064.0,4.0,2.0,2.0,312.0,148.0,2015.0,-37.56857,144.91694,15510.0
4549,h,S,South-Eastern Metropolitan,4,14.7,3167.0,4.0,2.0,2.0,581.0,NaN,NaN,-37.93550,145.09790,3692.0
4291,h,SP,Western Metropolitan,4,11.2,3042.0,4.0,2.0,2.0,650.0,150.0,1960.0,-37.74290,144.88800,2291.0


step1 defining the preprocessing steps

In [9]:
from sklearn.compose import ColumnTransformer # bundles preprocessing steps justlike pipline bundles preprocessing with modeling
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

preprocessing numerical data

In [10]:
numerical_transformer= SimpleImputer(strategy='constant')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

Bundling preprocessing for both categories

In [11]:
preprocessor = ColumnTransformer(
    transformers =[
        ('num', numerical_transformer,numerical_col),
        ('cat',categorical_transformer,categorical_col)
    ]
)

In [12]:
model = RandomForestRegressor(n_estimators=100, random_state=0)


In [13]:
pipline = Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',model)
])

In [14]:
pipline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Rooms', 'Distance',
                                                   'Postcode', 'Bedroom2',
                                                   'Bathroom', 'Car',
                                                   'Landsize', 'BuildingArea',
                                                   'YearBuilt', 'Lattitude',
                                                   'Longtitude',
                                                   'Propertycount']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('oneh

Note the validation data is automatically preprocessed and prediction made.

In [15]:
pred = pipline.predict(X_val)

In [16]:
print(mean_absolute_error(pred,y_val))

170141.86451399117
